# Homework 4 - Spark

In this homework, we are practicing Apache Spark.

You are required to turn in this notebook as BDM\_HW4\_Spark\_**NetId**.ipynb. You will be asked to complete each task using Apache Spark. Output can be printed in the notebook.

## Task 1 (5 points)

You are asked to implement Homework 3 using Spark. The description is provided below for your convenience.

You are asked to implement the Social Triangle example discussed in class. In particular, given the email dataset, please list all "reciprocal" relationships in the company. Recall that:

If A emails B and B emails A, then A and B is *reciprocal*.

If A emails B but B doesn’t email A, then A and B is *directed*.

**Dataset:** We will use a subset of the open [Enron Email Dataset](https://www.cs.cmu.edu/~./enron/ "Enron Email Dataset"), which contains approximately 10,000 simplified email headers from the Enron Corporation. You can download this dataset from NYU Classes as **enron_mails_small.csv**. The file contains 3 columns *Date*, *From*, and *To*. Their description is as follows:

|Column name|Description|
|--|--|
|Date |The date and time of the email, in the format YYYY-MM-DD hh-mm-ss, <br />e.g. "1998-10-30 07:43:00" |
|From |The sender email address, <br />e.g. "mark.taylor@enron.com" |
|To | A list of recipients' email addresses separated by semicolons ';', <br />e.g. "jennifer.fraser@enron.com;jeffrey.hodge@enron.com" |

Note that, we only care about users employed by Enron, or only relationships having email addresses that end with *'@enron.com'*.

The expected output is also provided below. For each reciprocal relationship, please output a tuple consisting of two strings. The first one is always **'reciprocal'**. And the second one is a string showing the name of the two person in the following format: **'Jane Doe : John Doe'**. The names should be presented in the lexical order, i.e. there will not be a 'John Doe : Jane Doe' since 'Jane' is ordered before 'John.

Though the dataset only contains email addresses, not actual names, we're assuming that the email aliases were created based on their name. For example:

|Email Address|Converted Name|
|--|--|
|mark.taylor@enron.com|Mark Taylor|
|alan.aronowitz@enron.com|Alan Aronowitz|
|marc.r.cutler@enron.com|Marc R Cutler|
|hugh@enron.com|Hugh|

Please fill the code block with a series of MapReduce jobs using your own mapper and reducer functions. Be sure to include the naming convention logic into one of your mappers and/or reducers.

In [2]:
Email_FN = "enron_mails_small.csv"

In [3]:
email = sc.textFile(Email_FN)
list(enumerate(email.first().split(",")))

[(0, 'Date'), (1, 'From'), (2, 'To')]

In [4]:
def Capitalize(full_name):
    return ' '.join([name.capitalize() for name in full_name.split(' ')])

def extractRelations(partitionID, rows):
    if partitionID == 0:  # 去掉第一行属性名
        next(rows)
    import csv
    reader = csv.reader(rows)
    for row in reader:
        if 'enron.com' in row[1]:
            Sender = row[1].split('@')[0].replace('.', ' ')
            Recipients = [recipient.split('@')[0].replace('.', ' ') for recipient in row[2].split(';') 
                          if 'enron.com' in recipient]
            if len(Recipients) > 0:
                for Recipient in Recipients:
                    yield (Capitalize(Sender), Capitalize(Recipient))

emailRelations = email.mapPartitionsWithIndex(extractRelations)
emailRelations.take(5)

[('Mark Taylor', 'Shari Stack'),
 ('Mark Taylor', 'Yao Apasu'),
 ('Mark Taylor', 'Paul Simons'),
 ('Mark Taylor', 'Justin Boyd'),
 ('Mark Taylor', 'Tana Jones')]

In [5]:
emailRelations = emailRelations.mapValues(lambda x: {x}).reduceByKey(lambda x, y: x.union(y))
emailRelations.take(1)

[('Mark Haedicke',
  {'Alan Aronowitz',
   'Anita Fam',
   'Barbara Gray',
   'Bob Crane',
   'Brad Richter',
   'Brent Enron',
   'Bryan Burnett',
   'Chris Gaffney',
   'Christi Nicolay',
   'Christian Yoder',
   'Cliff Baxter',
   'Cynthia Sandherr',
   'Dan Lyons',
   'Darlene Forsyth',
   'David Delainey',
   'David Forster',
   'David Oxley',
   'David Roland',
   'Deb Korkmas',
   'Don Black',
   'Dusty Paez',
   'Edward Ondarza',
   'Elisabeth Mccabe',
   'Elizabeth Grant',
   'Elizabeth Labanowski',
   'Elizabeth Sager',
   'Fiona Lavelle',
   'G Aarti',
   'Gareth Bahlmann',
   'George Mcclellan',
   'Greg Johnston',
   'Greg Piper',
   'Greg Whalley',
   'Jaime Alatorre',
   'James Fallon',
   'James Grace',
   'Janette Elbertson',
   'Janice Moore',
   'Jeffrey Hodge',
   'Jeffrey Keeler',
   'John Ale',
   'John Sherriff',
   'Joseph Hirl',
   'Julia Murray',
   'Julie Armstrong',
   'Justin Boyd',
   'Kenneth Rice',
   'Kevin Hannon',
   'Kristin Armstrong',
   'Kristina 

In [6]:
def rewriteRelations(Sender, Recipients): 
    for Recipient in Recipients:
        relation = sorted([Sender, Recipient])
        yield (relation[0] + " : " + relation[1], 1)

emailRelations = emailRelations.flatMap(lambda x: rewriteRelations(x[0], x[1]))
emailRelations.take(5)

[('Mark Haedicke : Mark Holsworth', 1),
 ('Greg Whalley : Mark Haedicke', 1),
 ('Lisa Mellencamp : Mark Haedicke', 1),
 ('Mark Haedicke : Monica Jordan', 1),
 ('Mark Haedicke : Tim Dorsey', 1)]

In [7]:
from operator import add

emailRelations = emailRelations.reduceByKey(add) \
    .filter(lambda x: x[1] == 2).sortByKey().map(lambda x: ('reciprocal', x[0])).collect()
print(len(emailRelations))
emailRelations

35


[('reciprocal', 'Brenda Whitehead : Elizabeth Sager'),
 ('reciprocal', 'Carol Clair : Debra Perlingiere'),
 ('reciprocal', 'Carol Clair : Mark Taylor'),
 ('reciprocal', 'Carol Clair : Richard Sanders'),
 ('reciprocal', 'Carol Clair : Sara Shackleton'),
 ('reciprocal', 'Carol Clair : Tana Jones'),
 ('reciprocal', 'Debra Perlingiere : Kevin Ruscitti'),
 ('reciprocal', 'Drew Fossum : Susan Scott'),
 ('reciprocal', 'Elizabeth Sager : Janette Elbertson'),
 ('reciprocal', 'Elizabeth Sager : Mark Haedicke'),
 ('reciprocal', 'Elizabeth Sager : Mark Taylor'),
 ('reciprocal', 'Elizabeth Sager : Richard Sanders'),
 ('reciprocal', 'Eric Bass : Susan Scott'),
 ('reciprocal', 'Fletcher Sturm : Greg Whalley'),
 ('reciprocal', 'Fletcher Sturm : Sally Beck'),
 ('reciprocal', 'Gerald Nemec : Susan Scott'),
 ('reciprocal', 'Grant Masson : Vince Kaminski'),
 ('reciprocal', 'Greg Whalley : Richard Sanders'),
 ('reciprocal', 'Janette Elbertson : Mark Taylor'),
 ('reciprocal', 'Janette Elbertson : Richard Sa

## Task 2 (5 points)

You are asked to implement Task 2 of Lab 5. The description is provided below for your convenience.

We’ll be using two NYC open data sets: the SAT Results and the NYC High School Directory data sets. Both can be downloaded from the links below, or from online class resources.

**Dataset**: *Please note that each school is uniquely identified by an DBN code, which should be found on both data sets.*

**SAT_Results.csv**
Source: https://nycopendata.socrata.com/Education/SAT-Results/f9bf-2cp4  
Description: “The most recent school level results for New York City on the SAT. Results are available at the school level for the graduating seniors of 2012.”

**DOE_High_School_Directory_2014-2015.csv**
Source: https://data.cityofnewyork.us/Education/DOE-High-School-Directory-2014-2015/n3p6-zve2  
Description: “Directory of NYC High Schools.”

We would like to know how the Math scores vary across bus lines or subway lines serving the schools. Your task is to compute the average Math scores of all schools along each bus line and subway line. You can find the bus and subway lines serving each school in the High School Dictionary as bus and subway columns.

The expected results are two lists:
1. A list of key/value pairs: with bus line as keys, and the average Math scores as values.
2. A list of key/value pairs: with subway line as keys, and the average Math scores as values.

In [9]:
SAT_FN = "SAT_Results.csv"
HSD_FN = "DOE_High_School_Directory_2014-2015.csv"

In [10]:
dfScores = spark.read.load(SAT_FN, format='csv', header=True, inferSchema=True)
dfScores = dfScores.select("DBN", 
                           dfScores['`SAT Math Avg. Score`'].cast('int').alias('mathscore'), 
                           dfScores['Num of SAT Test Takers'].cast('int').alias('ntakers')).na.drop()
dfScores = dfScores.select("DBN", (dfScores.mathscore * dfScores.ntakers).alias("total"), "ntakers")
dfScores.head()

Row(DBN='02M047', total=6400, ntakers=16)

In [11]:
dfSchools = spark.read.load(HSD_FN, format="csv", header=True, inferSchema=True)
dfSchools = dfSchools.select("dbn", "bus", "subway")
dfSchools.head()

Row(dbn='01M292', bus='B39, M14A, M14D, M15, M15-SBS, M21, M22, M9', subway='B, D to Grand St ; F to East Broadway ; J, M, Z to Delancey St-Essex St')

In [12]:
dfResults = dfSchools.join(dfScores, dfSchools.dbn == dfScores.DBN, how="inner")
dfResults.head()

Row(dbn='01M292', bus='B39, M14A, M14D, M15, M15-SBS, M21, M22, M9', subway='B, D to Grand St ; F to East Broadway ; J, M, Z to Delancey St-Essex St', DBN='01M292', total=11716, ntakers=29)

In [13]:
import pyspark.sql.functions as f

dfResults.withColumn("BUS", f.explode(f.split("bus", ", "))).head(5)

[Row(dbn='01M292', BUS='B39', subway='B, D to Grand St ; F to East Broadway ; J, M, Z to Delancey St-Essex St', DBN='01M292', total=11716, ntakers=29),
 Row(dbn='01M292', BUS='M14A', subway='B, D to Grand St ; F to East Broadway ; J, M, Z to Delancey St-Essex St', DBN='01M292', total=11716, ntakers=29),
 Row(dbn='01M292', BUS='M14D', subway='B, D to Grand St ; F to East Broadway ; J, M, Z to Delancey St-Essex St', DBN='01M292', total=11716, ntakers=29),
 Row(dbn='01M292', BUS='M15', subway='B, D to Grand St ; F to East Broadway ; J, M, Z to Delancey St-Essex St', DBN='01M292', total=11716, ntakers=29),
 Row(dbn='01M292', BUS='M15-SBS', subway='B, D to Grand St ; F to East Broadway ; J, M, Z to Delancey St-Essex St', DBN='01M292', total=11716, ntakers=29)]

In [14]:
dfBus = dfResults.withColumn("BUS", f.explode(f.split("bus", ", "))).groupBy("bus").sum("total", "ntakers")
dfBus = dfBus.withColumn('avg', dfBus[1] / dfBus[2])
dfBus = dfBus.select("BUS", dfBus['avg'].cast('int'))
dfBus = dfBus.sort('avg', ascending=False)
dfBus.show()

+-----+---+
|  BUS|avg|
+-----+---+
|S1115|612|
|  M79|594|
|  Q42|582|
|  M22|574|
|  Bx3|571|
|  B52|560|
|  B63|557|
|  B69|548|
|  B54|543|
|  B25|541|
|  M20|540|
|   M9|539|
|  B65|538|
|  M86|538|
|  B45|534|
| Bx10|534|
| Bx26|533|
| B103|531|
|  Q64|529|
| Bx22|525|
+-----+---+
only showing top 20 rows



In [15]:
dfSub = dfResults.withColumn("Sub", f.explode(f.split("subway", ", "))).groupBy("Sub").sum("total", "ntakers")
dfSub = dfSub.withColumn('avg', dfSub[1] / dfSub[2])
dfSub = dfSub.select("Sub", dfSub['avg'].cast('int'))
dfSub = dfSub.sort('avg', ascending=False)
dfSub.show()

+--------------------+---+
|                 Sub|avg|
+--------------------+---+
|E to Chambers St ...|735|
|4 to Bedford Park...|683|
|     SIR to New Dorp|682|
|Z to Jamaica Cent...|660|
|5 to Nevins St ; ...|659|
|N to Atlantic Ave...|659|
|R to DeKalb Ave ;...|659|
|   D to 145th St ; B|654|
|        6 to 77th St|594|
|D to Bedford Park...|593|
|1 to 66th St - Li...|575|
|    R to 36th St ; N|568|
|    R to 23rd St ; B|563|
|                   3|547|
|Q to 14th St-Unio...|538|
|S to Beach 105th ...|537|
|                   A|534|
|        R to 23rd St|533|
|                   1|532|
|                   Q|531|
+--------------------+---+
only showing top 20 rows

